In [ ]:
#import os

# args.output_attention = True

#exp = Exp(args)

#model = exp.model

#setting = 'ETTh1_96_24_Autoformer_ETTh1_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0'
#path = os.path.join(args.checkpoints,setting,'checkpoint.pth')
#model.load_state_dict(torch.load(path))

In [ ]:
import torch
import os
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name())
print(torch.cuda.current_device())
torch.cuda.set_device(0)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = '0' #disable CUDA by setting CUDA_VISIBLE_DEVICES to an empty string 


In finance and stock market analysis, it's not uncommon to consider a small percentage change as "neutral" or "no change" because daily stock prices can fluctuate due to a variety of factors, including market volatility. By setting a threshold of 0.25%, you're essentially saying that any change in price less than this percentage is insignificant and should be considered neutral. 

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

commodities = ['gold', 'silver', 'natural_gas', 'coffee', 'corn', 'crude_oil']
OT_dict = {0: 'Decrease', 1: 'Increase'}

fig, axes = plt.subplots(nrows=1, ncols=len(commodities), figsize=(20, 5))


for i, commodity in enumerate(commodities):
    df = pd.read_csv(f'./dataset/commodity/{commodity}_data.csv')

    df['date'] = pd.to_datetime(df['date'])

    # Make sure the 'close' column is correctly named
    df.columns = df.columns.str.replace('OT', 'close')

    # Calculate the percentage change between the closing price of the next day and the current day
    df['close_pct_change'] = df['close'].pct_change(periods=-1) * 100
    
    # Create the target column based on the 'close_pct_change' column
    df['OT'] = 0.0 # neutral
    df.loc[df['close_pct_change'] > 0, 'OT'] = 1.0 # increase
    # df.loc[df['close_pct_change'] < -0.25, 'OT'] = 2 # decrease

    df = df.drop(columns=['close_pct_change'])
    df = df.drop(columns=['close', 'Adj Close'])

    # Drop the last row as it doesn't have a valid 'OT' value
    df = df[:-1]

    # Save the dataframe as a new CSV file
    df.to_csv(f'./dataset/commodity/{commodity}_data_c.csv')
    
    sns.countplot(x='OT', data=df, ax=axes[i])
    axes[i].set_xticks([0, 1])
    axes[i].set_xticklabels([OT_dict[i] for i in [0, 1]])
    axes[i].set_title(f'Distribution of {commodity} classes')

# Displaying the plots
plt.tight_layout()
plt.savefig('classification_bar.png')
plt.show()

In [ ]:
import pandas as pd
df = pd.read_csv(f'./dataset/commodity/gold_data_c.csv')
df.info()

In [1]:
# D Linear

import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

models = ['Autoformer', 'Informer', 'Transformer', 'DLinear', 'NLinear']
data_paths = ['gold_data_c.csv', 'silver_data_c.csv', 'crude_oil_data_c.csv', 'natural_gas_data_c.csv', 'corn_data_c.csv', 'coffee_data_c.csv']
label_lens = [14, 28, 42, 56]
pred_lens = [56]
args = dotdict()

for data_path in data_paths:
    for pred_len in pred_lens:

        # basic config
        args.is_training = 1
        args.model_id = ''
        args.model = 'DLinear' #Autoformer/Informer/Transformer/DLinear/NLinear

        # data loader
        args.data = 'custom'
        args.root_path = './dataset/commodity/'
        args.data_path = data_path
        args.features = 'M'
        args.target = 'OT'
        args.freq = 'd'
        args.checkpoints = './checkpoints/'

        # forecasting task
        args.seq_len = 96
        args.label_len = pred_len
        args.pred_len = pred_len

        # DLinear
        args.individual = False

        # Formers 
        args.embed_type = 0
        args.enc_in = 6
        args.dec_in = 6
        args.c_out = 6
        args.d_model = 512
        args.n_heads = 8
        args.e_layers = 2
        args.d_layers = 1
        args.d_ff = 2048
        args.moving_avg = 25
        args.factor = 3
        args.distil = True
        args.dropout = 0.05
        args.embed = 'timeF'
        args.do_predict = True

        # optimization
        args.num_workers = 10
        args.itr = 1
        args.train_epochs = 50
        args.patience = 5
        args.learning_rate = 0.0005
        args.batch_size = 8
        args.lradj = 'type1'
        args.des = 'Exp'

        # GPU
        args.gpu = 0
        args.devices = '0'
        args.use_gpu = True
        args.use_multi_gpu = False

        # print('Args in experiment:')
        # print(args)

        Exp = Exp_Main
        print(f'######################  {data_path}_{args.model}_{pred_len}_{pred_len}  ######################')
        if args.is_training:
            for ii in range(args.itr):
                # setting record of experiments
                setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
                    args.data_path.split("_")[0],
                    args.model_id,
                    args.model,
                    args.data,
                    args.features,
                    args.seq_len,
                    args.label_len,
                    args.pred_len,
                    args.d_model,
                    args.n_heads,
                    args.e_layers,
                    args.d_layers,
                    args.d_ff,
                    args.factor,
                    args.embed,
                    args.distil,
                    args.des, ii)

                exp = Exp(args)  # set experiments
                print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                exp.train(setting)

                print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                exp.test(setting)

                torch.cuda.empty_cache()


######################  gold_data_c.csv_DLinear_56_56  ######################
Use GPU: cuda:0
>>>>>>>start training : gold__DLinear_custom_ftM_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1608
val 197
test 447
	iters: 100, epoch: 1 | loss: 0.4515233
	speed: 0.2572s/iter; left time: 2559.4394s
	iters: 200, epoch: 1 | loss: 0.5054529
	speed: 0.0026s/iter; left time: 25.7438s
Epoch: 1 cost time: 26.88035297393799
Epoch: 1, Steps: 201 | Train Loss: 0.4764939 Vali Loss: 1.4964520 Test Loss: 0.9893274
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.4311605
	speed: 0.3708s/iter; left time: 3615.2854s
	iters: 200, epoch: 2 | loss: 0.5266655
	speed: 0.0022s/iter; left time: 21.6547s
Epoch: 2 cost time: 24.580622673034668
Epoch: 2, Steps: 201 | Train Loss: 0.4432168 Vali Loss: 1.3520074 Test Loss: 0.8852887
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 0.4864195
	speed: 0.3779s/iter; left time: 3608.8218s
	ite

accuracy, f1: 0.744 & 0.709 
440, 440
######################  crude_oil_data_c.csv_DLinear_56_56  ######################
Use GPU: cuda:0
>>>>>>>start training : crude__DLinear_custom_ftM_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1609
val 197
test 448
	iters: 100, epoch: 1 | loss: 0.3636216
	speed: 0.2346s/iter; left time: 2334.2418s
	iters: 200, epoch: 1 | loss: 0.4252815
	speed: 0.0023s/iter; left time: 22.2039s
Epoch: 1 cost time: 24.502159595489502
Epoch: 1, Steps: 201 | Train Loss: 0.3394222 Vali Loss: 0.7322459 Test Loss: 0.3920832
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.4000925
	speed: 0.3790s/iter; left time: 3694.8723s
	iters: 200, epoch: 2 | loss: 0.3211789
	speed: 0.0022s/iter; left time: 21.1756s
Epoch: 2 cost time: 24.31145691871643
Epoch: 2, Steps: 201 | Train Loss: 0.3158242 Vali Loss: 0.7225709 Test Loss: 0.3844586
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 0.2705851
	spe

Epoch: 8 cost time: 26.886470317840576
Epoch: 8, Steps: 201 | Train Loss: 0.4725164 Vali Loss: 0.5384447 Test Loss: 1.9978476
EarlyStopping counter: 4 out of 5
Updating learning rate to 3.90625e-06
	iters: 100, epoch: 9 | loss: 0.4365718
	speed: 0.3869s/iter; left time: 3227.5665s
	iters: 200, epoch: 9 | loss: 0.5031409
	speed: 0.0022s/iter; left time: 17.7573s
Epoch: 9 cost time: 24.930137872695923
Epoch: 9, Steps: 201 | Train Loss: 0.4723713 Vali Loss: 0.5414367 Test Loss: 1.9970576
EarlyStopping counter: 5 out of 5
Early stopping
>>>>>>>testing : natural__DLinear_custom_ftM_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
test 448
accuracy, f1: 0.720 & 0.685 
448, 448
######################  corn_data_c.csv_DLinear_56_56  ######################
Use GPU: cuda:0
>>>>>>>start training : corn__DLinear_custom_ftM_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1606
val 197
test 447
	iters: 1

In [ ]:
import pandas as pd
exp.args.root_path = './dataset/commodity/'
exp.args.data_path = 'gold_data.csv'

df = pd.read_csv(os.path.join(args.root_path, args.data_path))
df.head()
df.info()


In [ ]:
# df['date'] = pd.to_datetime(df['date'])
# # df.to_csv(os.path.join(args.root_path, args.data_path), index=False) 
# last_date = df['date'].max()
# new_rows = pd.DataFrame(columns=df.columns, index=range(1, 31))
# new_rows['date'] = pd.date_range(start=last_date + pd.DateOffset(1), periods=30)
# df = pd.concat([df, new_rows], sort=False)
# df.tail()



In [ ]:
if args.do_predict:
    print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    prediction=exp.predict(setting, True)
    prediction = np.load('./results/'+setting+'/real_prediction.npy')

    prediction.shape


In [ ]:
from data_provider.data_loader import Dataset_Pred
from torch.utils.data import DataLoader
from data_provider.data_loader import Dataset_Custom
Data = Dataset_Custom

freq = 'b'

Data = Dataset_Custom
timeenc = 0 if args.embed!='timeF' else 1
flag = 'test'; shuffle_flag = False; drop_last = True; batch_size = 1

data_set = Data(
    root_path=args.root_path,
    data_path=args.data_path,
    flag=flag,
    size=[args.seq_len, args.label_len, args.pred_len],
    features=args.features,
    timeenc=timeenc,
    target=args.target, # HULL here
    freq=args.freq # 'h': hourly, 't':minutely
)
data_loader = DataLoader(
    data_set,
    batch_size=batch_size,
    shuffle=shuffle_flag,
    num_workers=args.num_workers,
    drop_last=drop_last)

print(len(data_set), len(data_loader))


In [ ]:
exp.predict(setting, data_loader)

In [ ]:
# When we finished exp.train(setting) and exp.test(setting), we will get a trained model and the results of test experiment
# The results of test experiment will be saved in ./results/{setting}/pred.npy (prediction of test dataset) and ./results/{setting}/true.npy (groundtruth of test dataset)

preds = np.load('./results/'+setting+'/pred.npy')
trues = np.load('./results/'+setting+'/true.npy')

# [samples, pred_len, dimensions]
print(preds.shape, trues.shape)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch.nn.functional as F
import torch
import matplotlib.pyplot as plt 
import numpy as np
import matplotlib
data_pred = np.load('./results/'+setting+'/real_prediction.npy')
data_pred = torch.from_numpy(data_pred).permute(0,2,1)

plt.figure()
print(data_pred.shape)
#预测OT
plt.plot(data_pred[-1,-1,:])
print(data_pred[-1,-1,:].shape)
plt.show()
plt.plot(data_pred[0,-1,:])
print(data_pred[0,-1,:].shape)
#plt.show()
# draw HUFL prediction
plt.plot(data_pred[0,0,:])
print(data_pred[-1,-1,:].shape)
plt.show()
'''
Ground Truth
'''
data_gt = np.load('./results/'+setting+'/true.npy')
data_gt = torch.from_numpy(data_gt).permute(0,2,1)

# OT
plt.plot(data_gt[-1,-1,:])
print(data_gt[-1,-1,:].shape)
plt.show()
plt.plot(data_gt[0,-1,:])
print(data_gt[0,-1,:].shape)
#plt.show()
# draw HUFL prediction
plt.plot(data_gt[0,0,:])
print(data_gt[-1,-1,:].shape)
plt.show()



In [7]:
setting = 'coffee_classification_Autoformer_custom_ftS_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0'

In [8]:
if (args.model == 'DLinear'):
    model_name = setting
    for root, dirs, files in os.walk("checkpoints"):
        for name in files:
            model_path = os.path.join(root, name)
            if model_name not in model_path:
                continue
            weights = torch.load(model_path,map_location=torch.device('cpu'))
            weights_list = {}
            weights_list['seasonal'] = weights['Linear_Seasonal.weight'].numpy()
            weights_list['trend'] = weights['Linear_Trend.weight'].numpy()

            #print(root)
            #print(root.split('/'))

            save_root = 'weights_plot/%s'%root.split('\\')[1] #changed from save_root = 'weights_plot/%s'%root.split('/')[1]
            if not os.path.exists('weights_plot'):
                os.mkdir('weights_plot')
            if not os.path.exists(save_root):
                os.mkdir(save_root)
        
            for w_name,weight in weights_list.items():
                fig,ax=plt.subplots()
                im=ax.imshow(weight,cmap='plasma_r')
                fig.colorbar(im,pad=0.03)
                plt.savefig(os.path.join(save_root,w_name + '.pdf'),dpi=500)
                plt.show()
                plt.close()

KeyError: 'Linear_Seasonal.weight'